In [1]:
#stochasics within NYSE
###would be great for indices like SPY,XLK,GDXJ,OIH,etc.

In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
#RSI and Stochastic Oscillator 
#The myRSI() or RSI() because i tweaked the equation to take into effect overall returns for each day
def myRSI(ticker):
    '''RSI can be used to see if a security is overbought
    or oversold. If the number is 30 or below, the stock 
    is believed to be oversold and this could be a buy
    signal. If it is 70-100, this is overbought and it could
    be a sell or short signal'''
    end = dt.datetime.now()
    # a year of trading
    start = end - dt.timedelta(days=365)
    # gaining the data from yahoo finance
    # ticker should be included as an argument when the function is called
    data = web.DataReader(ticker,'yahoo',start,end)
    # of high, low, close, adj close, and volume given, I only need adjusted close 
    df = data[['Adj Close']]
    # create a column called daily return
    df['Daily Return'] = df['Adj Close'].pct_change()*100
    # dataframe consists of 14 days because most RSI's are calculated in a 14 trading day span (15 for calculation purposes)
    df = df.iloc[-14:]
    # making a dataframe only for the percent return of each day
    df1 = df['Daily Return']
    # find days with positive and negative returns
    gain = df1[df1 > 0]
    loss = df1[df1 < 0]
    # average of all the days that were positive or negative 
    avgGain = (gain.sum()/14)
    avgLoss = (loss.sum()/14)
    RS = abs(avgGain/avgLoss)
    RSI = 100 - (100 / (1 + RS))
    #print(f'{ticker} has a myRSI: {RSI}')
    return RSI
    
### Standard RSI ###
def RSI(ticker):
    '''This is the traditional RSI where it only matters if
    a day experiences a positive or negative daily return.
    myRSI() differs by adding in a the fact that returns
    based on percentages (not booleans (pos or neg)) have an
    effect on the overall equation.'''
    end = dt.datetime.now()
    # a year of trading
    start = end - dt.timedelta(days=365)
    # gaining the data from yahoo finance
    # ticker should be included as an argument when the function is called
    data = web.DataReader(ticker,'yahoo',start,end)
    # of high, low, close, adj close, and volume given, I only need adjusted close 
    df = data[['Adj Close']]
    # create a column called daily return
    df['Daily Return'] = df['Adj Close'].pct_change()*100
    # dataframe consists of 14 days because most RSI's are calculated in a 14 trading day span (15 for calculation purposes)
    df = df.iloc[-14:]
    # making a dataframe only for the percent return of each day
    df['Up/Down'] = np.sign(df['Daily Return'])
    df1 = df['Up/Down']
    # find days with positive and negative returns
    gain = df1[df1 > 0]
    loss = df1[df1 < 0]
    # average of all the days that were positive or negative 
    avgGain = (gain.sum()/14)
    avgLoss = abs(loss.sum()/14)
    RS = (avgGain/avgLoss)
    RSI = 100 - (100 / (1 + RS))
    #print(f'{ticker} has an RSI of {RSI}')
    return RSI

In [3]:
myRSI('SPY')

52.41393999660634

In [4]:
def stochRSI(ticker,period):
    '''Input ticker('SPY') and period (amount of days) 
    Below 30 (50 for me) is a buy signal and above 80 is a sell or short signal
    When it is below 30, it is oversold, and when it is above 70, it is overbought'''
    end = dt.datetime.now()
    # print out the shape of the dataframe
    start = end - dt.timedelta(days=101)
    # gaining the data from yahoo finance
    # ticker should be included as an argument when the function is called
    data = web.DataReader(ticker,'yahoo',start,end)
    # of high, low, close, adj close, and volume given, I only need adjusted close 
    df = data[['Adj Close']]
    # create a column called daily return
    df['Daily Return'] = df['Adj Close'].pct_change()*100
    # making a dataframe only for the percent return of each day
    df['Up/Down'] = np.sign(df['Daily Return'])
    #drop the nan values
    df.dropna(inplace=True)
    # the returns that were up => display a 1 in the 'Up' Column
    df['Up']=(df['Up/Down']>0).astype(int)
    # the returns that were down => display a 1 in the 'Down' Column
    df['Down']=(df['Up/Down']<0).astype(int)
    ###create arrays for up and down columns to compute a rolling 14 day RSI###
    #could've done a for loop, but this was easier for me
    up = df['Up']
    down = df['Down']
    gain = pd.Series(up).rolling(window=period).sum()
    loss = pd.Series(down).rolling(window=period).sum()
    df['Gain'] = gain/period
    df['Loss'] = loss/period
    #create column for RS
    df['RS'] = df['Gain']/df['Loss']
    #create column for RSI
    df['RSI'] = 100 - (100/(1+df['RS']))
    #drop nan for no reason but I have ocd
    df.dropna(inplace=True)
    #trading period... 14 is standard but can try longer (might be better for most stocks I trade)
    df1 = df.iloc[-period:]
    maxRSI = df1['RSI'].max()
    minRSI = df1['RSI'].min()
    currentRSI = df1['RSI'][-1]
    RSIstoch = ((currentRSI - minRSI)/(maxRSI-minRSI))*100
    print(f'The current situation for {ticker} in a {period} day RSI Stochastic: {RSIstoch}')
    #test to see if data is correct / accurate
    print(df1)

### visualRSI is for dropping the RSI's into an excel file to see patterns ###    
    
def visualRSI():
    '''Coolest function... Can see the RSI values fluctuate (usually) from 20/30 to 70/80/90.
    Stocks that fluctuate and have good credit (or part of big indices) would be good picks 
    for trading strategy... below 50 is a buy indicator'''
    end = dt.datetime.now()
    # print out the shape of the dataframe
    start = end - dt.timedelta(days=365*3)
    # gaining the data from yahoo finance
    # ticker should be included as an argument when the function is called
    stock_list = ['SPY','QQQ','SQQQ','IMOEX.ME','^N100','^N225','^STOXX','NLY', 'NRZ', 'AAPL', 'MSFT', 'XLK', 'AUY', 'GDXJ','GDX', 'AGNC', 'ANH', 'TWO', 'IVR','ARR', 'CIM']
    stocks = []
    for ticker in stock_list:
        df = web.DataReader(ticker,'yahoo',start,end)
        df = df[['Adj Close']]
        df[f'{ticker} Adj Close'] = df['Adj Close']
        # create a column called daily return
        df[f'{ticker} Daily Return'] = df[f'{ticker} Adj Close'].pct_change()*100
        # making a dataframe only for the percent return of each day
        df[f'{ticker} Up/Down'] = np.sign(df[f'{ticker} Daily Return'])
        #drop the nan values
        df.dropna(inplace=True)
        df[f'{ticker} Up']=(df[f'{ticker} Up/Down']>0).astype(int)
        df[f'{ticker} Down']=(df[f'{ticker} Up/Down']<0).astype(int)
        up = df[f'{ticker} Up']
        down = df[f'{ticker} Down']
        gain = pd.Series(up).rolling(window=14).sum()
        loss = pd.Series(down).rolling(window=14).sum()
        df[f'{ticker} Gain'] = gain/14
        df[f'{ticker} Loss'] = loss/14
        df[f'{ticker} RS'] = df[f'{ticker} Gain']/df[f'{ticker} Loss']
        df[f'{ticker} RSI'] = 100 - (100/(1+df[f'{ticker} RS']))
        df.dropna(inplace=True)
        stocks.append(df)
    data = pd.concat(stocks, axis=1)
    data = data.reset_index()
    drop = data.iloc[:,1::10]
    data.drop(drop, inplace=True, axis=1)
    data.dropna(inplace=True)
    df1 = data[['Date','SPY Daily Return','SPY RSI','QQQ Daily Return','QQQ RSI','SQQQ Daily Return','SQQQ RSI','IMOEX.ME Daily Return','IMOEX.ME RSI','^N100 Daily Return','^N100 RSI','^N225 Daily Return','^N225 RSI','^STOXX Daily Return','^STOXX RSI','NLY Daily Return','NLY RSI','NRZ Daily Return','NRZ RSI','AAPL Daily Return','AAPL RSI','MSFT Daily Return','MSFT RSI','XLK Daily Return','XLK RSI','AUY Daily Return','AUY RSI','GDXJ Daily Return','GDXJ RSI','GDX Daily Return','GDX RSI','AGNC Daily Return','AGNC RSI','ANH Daily Return','ANH RSI','TWO Daily Return','TWO RSI','IVR Daily Return','IVR RSI','ARR Daily Return','ARR RSI','CIM Daily Return','CIM RSI']]
    #return RSI to an excel sheet to vizualize the RSI moving back and forth from below 50 to above 50
    df1.to_excel(f'StochasticRSIvisualization.xlsx')

In [14]:
visualRSI()

In [5]:
stochRSI('QQQ',28)

The current situation for QQQ in a 28 day RSI Stochastic: 100.0
             Adj Close  Daily Return  Up/Down  Up  Down      Gain      Loss  \
Date                                                                          
2020-04-06  196.479996      7.149480      1.0   1     0  0.464286  0.535714   
2020-04-07  196.399994     -0.040718     -1.0   0     1  0.464286  0.535714   
2020-04-08  200.570007      2.123225      1.0   1     0  0.464286  0.535714   
2020-04-09  200.860001      0.144585      1.0   1     0  0.464286  0.535714   
2020-04-13  203.029999      1.080354      1.0   1     0  0.500000  0.500000   
2020-04-14  211.860001      4.349112      1.0   1     0  0.500000  0.500000   
2020-04-15  209.429993     -1.146988     -1.0   0     1  0.500000  0.500000   
2020-04-16  213.250000      1.824002      1.0   1     0  0.535714  0.464286   
2020-04-17  215.289993      0.956621      1.0   1     0  0.571429  0.428571   
2020-04-20  212.740005     -1.184443     -1.0   0     1  0.535714  